In [1]:
import sys
if 'google.colab' in sys.modules:
    print("Running on Google Colab")
    ## The Dataset libraries make it possible to load and process datasets from the Hugging Face hub
    !pip install datasets
    !pip install rich
    !pip install seaborn
    !pip install imagemagick
    !pip install optuna
    !pip install autogluon==0.6.2
    !pip install -U pip setuptools wheel
    !pip install -U 'spacy[cuda-autodetect]'
    !python -m spacy download en_core_web_sm
    !pip bitsandbytes
    !pip install accelerate
else:
    print("Not running on Google Colab")
from rich import print
import torch

Not running on Google Colab


/home/null/miniconda3/envs/ais_workshop/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Donload  the Flan T5 model from Hugging Face hub
1. The t5 model base is 1GB in size. So, it will take some time to download the model.


In [32]:
import time 
import torch
from transformers import AutoModelForSeq2SeqLM
from huggingface_hub import HfApi
## check if the system has cuda devices
if torch.cuda.is_available():
    print("CUDA is available")
    # pip install bitsandbytes accelerate
    from transformers import T5Tokenizer, T5ForConditionalGeneration

    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto", load_in_8bit=True)

    input_text = "translate English to German: How old are you?"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    start_time = time.time()
    outputs = model.generate(input_ids)
    print(tokenizer.decode(outputs[0]))
    print("Time taken: ", time.time() - start_time)
else:
    print("CUDA is not available")

    from transformers import T5Tokenizer, T5ForConditionalGeneration

    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

    input_text = "translate English to German: How old are you?"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    start_time = time.time()
    outputs = model.generate(input_ids)
    print(tokenizer.decode(outputs[0] , skip_special_tokens=True))

    print("Time taken: ", time.time() - start_time)


CUDA is not available

Wie ich er bitten?

Time taken:  0.6217010021209717

# Get the size of the model context lengths

In [36]:
print(f"The model has {model.num_parameters():,} parameters")
## get the context length
print(f"The context length is {model.config.n_positions}")

The model has 76,961,152 parameters

The context length is 512

In [46]:
# Template for the model
TEMPLATE:str = "What is the meaning of life?"
input_ids = tokenizer(TEMPLATE, return_tensors="pt").input_ids
print(input_ids)
## Constastive Seach
outputs = model.generate(
    input_ids , 
    do_sample=True, 
    max_length =  300, 
    top_k = 100,
    top_p=0.70, 
    num_return_sequences = 3
)
for output in outputs:
    print(tokenizer.decode(output , skip_special_tokens=True))

tensor([[ 363,   19,    8, 2530,   13,  280,   58,    1]])

life in which there is something in common and a place in life.

life is a kind of thing.

life in the sea